In [1]:
from pathlib import Path
import sys
import cobra
import os
import pandas as pd
import re

In [2]:
path_root = Path("C:/Users/tobyl/OneDrive - The University of Manchester/Bioinformatics Masters/Research Project 2/Pycomo/PyCoMo/src")
sys.path.append(str(path_root))
import pycomo


### Creating community model
1. Load the member models

In [3]:
# loading member models
initial_path = Path().absolute().parent

model_member_dir = initial_path / r"old_files/models_members_test/Toy"
named_models = pycomo.load_named_models_from_dir(model_member_dir)

In [4]:
# checking modesl that have been loaded
print(named_models)

{'Bacteroides_uniformis_ATCC_8492': <Model Bacteroides_uniformis_ATCC_8492 at 0x209b06d45c0>, 'Bifidobacterium_adolescentis_ATCC_15703': <Model Bifidobacterium_adolescentis_ATCC_15703 at 0x209bc25c1d0>, 'Faecalibacterium_prausnitzii_M21_2': <Model Faecalibacterium_prausnitzii_M21_2 at 0x209bc218110>}


In [5]:
# checking if I can edit the models
model1 = named_models['Escherichia_coli_UTI89_UPEC']
for reaction in model1.exchanges:
    print(reaction.name)
    print(reaction)
    print(reaction.bounds)

1,5-Diaminopentane exchange
EX_15dap(e): 15dap[e] --> 
(0.0, 1000.0)
2-Dehydro-3-deoxy-D-gluconate exchange
EX_2ddglcn(e): 2ddglcn[e] --> 
(0.0, 1000.0)
2-Hydroxyphenylacetate exchange
EX_2hyoxplac(e): 2hyoxplac[e] --> 
(0.0, 1000.0)
4-Aminobutanoate exchange
EX_4abut(e): 4abut[e] --> 
(0.0, 1000.0)
4-Hydroxybenzoate exchange
EX_4hbz(e): 4hbz[e] <=> 
(-1.0, 1000.0)
7-Dehydrochenodeoxycholate exchange
EX_7dhcdchol(e): 7dhcdchol[e] --> 
(0.0, 1000.0)
alpha,12alpha-Dihydroxy-7-oxo-5beta-cholanate exchange
EX_7ocholate(e): 7ocholate[e] --> 
(0.0, 1000.0)
chenodeoxycholate exchange
EX_C02528(e): C02528[e] --> 
(0.0, 1000.0)
L-cysteate exchange
EX_Lcyst(e): Lcyst[e] --> 
(0.0, 1000.0)
Acetate exchange
EX_ac(e): ac[e] <=> 
(-1.0, 1000.0)
Acetoacetate exchange
EX_acac(e): acac[e] --> 
(0.0, 1000.0)
Acetaldehyde exchange
EX_acald(e): acald[e] --> 
(0.0, 1000.0)
N-Acetyl-D-glucosamine exchange
EX_acgam(e): acgam[e] <=> 
(-1.0, 1000.0)
exchange reaction for Adenosine
EX_adn(e): adn[e] <=> 
(-1.0,

2. checking the objective function of the loaded models

In [5]:
# if all have a biomass objective function then it is working
for model in named_models.values():
    print(model.objective)

Maximize
1.0*biomass536 - 1.0*biomass536_reverse_50d4e
Maximize
1.0*biomass345 - 1.0*biomass345_reverse_e128f
Maximize
1.0*biomass224 - 1.0*biomass224_reverse_c48a3


In [6]:
# creating the single organism models in PyCoMo format
single_org_models = []
for name, model in named_models.items():
    print(name)
    single_org_model = pycomo.SingleOrganismModel(model, name)
    single_org_models.append(single_org_model)

Bacteroides_uniformis_ATCC_8492
Bifidobacterium_adolescentis_ATCC_15703
Faecalibacterium_prausnitzii_M21_2


3. Creating the community model

In [7]:
community_name = "ten_gutControl_model"
com_model_obj = pycomo.CommunityModel(single_org_models, community_name)

In [11]:
##### medium creation code


parent_path = Path().absolute().parent
diet_filepath = parent_path / r"diet_info/western_fluxes.tsv"
diet_df = pd.read_csv(diet_filepath, sep = "\t")
diet_df.loc[:, 'Reaction'] = diet_df['Reaction'].apply(lambda reaction: reaction
                                                       .replace("[", "(")
                                                       .replace("]", ")")
                                                       )
diet_reactionID_list = list(diet_df.loc[:, 'Reaction'])

# getting all the models in the community model
model_members = com_model_obj.member_models

#creating the medium with the diet values
medium_exchange_dict = {}
#for single_org_model in model_members:
#    model = single_org_model.model
    
## potential solution
for reaction in com_model_obj.model.exchanges:
    reaction_id = reaction.id
    medium_reaction_id = reaction_id.split('(')[0] + "_medium"
    if reaction_id in medium_exchange_dict.values():
        continue
    elif reaction_id in diet_reactionID_list:
        flux = diet_df.loc[diet_df['Reaction'] == reaction_id, 'Flux Value'].values
        medium_exchange_dict[medium_reaction_id] = flux
    else:
            medium_exchange_dict[medium_reaction_id] = 1e-6
print(medium_exchange_dict)
        
           
           
       

No community model generated yet. Generating now:
Note: no products in the objective function, adding biomass to it.


Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.
c:\Users\tobyl\miniconda3\envs\masterswork\Lib\site-packages\cobra\medium\boundary_types.py:97: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return most[0]
Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.
c:\Users\tobyl\miniconda3\envs\masterswork\Lib\site-packages\cobra\medium\boundary_types.py:97: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, in

Note: no products in the objective function, adding biomass to it.


Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.
c:\Users\tobyl\miniconda3\envs\masterswork\Lib\site-packages\cobra\medium\boundary_types.py:97: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return most[0]
Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.
c:\Users\tobyl\miniconda3\envs\masterswork\Lib\site-packages\cobra\medium\boundary_types.py:97: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, in

Ignoring reaction 'EX__4abut_medium' since it already exists.
Ignoring reaction 'EX_C02528_medium' since it already exists.
Ignoring reaction 'EX_ac_medium' since it already exists.
Ignoring reaction 'EX_acgal_medium' since it already exists.
Ignoring reaction 'EX_acgam_medium' since it already exists.
Ignoring reaction 'EX_ala_L_medium' since it already exists.
Ignoring reaction 'EX_amylopect900_medium' since it already exists.
Ignoring reaction 'EX_arab_L_medium' since it already exists.
Ignoring reaction 'EX_arabinogal_medium' since it already exists.
Ignoring reaction 'EX_arabttr_medium' since it already exists.
Ignoring reaction 'EX_asn_L_medium' since it already exists.
Ignoring reaction 'EX_btn_medium' since it already exists.
Ignoring reaction 'EX_ca2_medium' since it already exists.
Ignoring reaction 'EX_cd2_medium' since it already exists.
Ignoring reaction 'EX_cellb_medium' since it already exists.
Ignoring reaction 'EX_cgly_medium' since it already exists.
Ignoring reaction

Note: no products in the objective function, adding biomass to it.


Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.
c:\Users\tobyl\miniconda3\envs\masterswork\Lib\site-packages\cobra\medium\boundary_types.py:97: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return most[0]
Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.
c:\Users\tobyl\miniconda3\envs\masterswork\Lib\site-packages\cobra\medium\boundary_types.py:97: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, in

In [9]:
# calling the community model to make it
com_model_obj.model

No community model generated yet. Generating now:
Note: no products in the objective function, adding biomass to it.


Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.
c:\Users\tobyl\miniconda3\envs\masterswork\Lib\site-packages\cobra\medium\boundary_types.py:97: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return most[0]
Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.
c:\Users\tobyl\miniconda3\envs\masterswork\Lib\site-packages\cobra\medium\boundary_types.py:97: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, in

Note: no products in the objective function, adding biomass to it.


Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.
c:\Users\tobyl\miniconda3\envs\masterswork\Lib\site-packages\cobra\medium\boundary_types.py:97: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return most[0]
Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.
c:\Users\tobyl\miniconda3\envs\masterswork\Lib\site-packages\cobra\medium\boundary_types.py:97: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, in

Ignoring reaction 'EX_h_medium' since it already exists.
Ignoring reaction 'EX_acald_medium' since it already exists.
Ignoring reaction 'EX_acgam_medium' since it already exists.
Ignoring reaction 'EX_ac_medium' since it already exists.
Ignoring reaction 'EX_adocbl_medium' since it already exists.
Ignoring reaction 'EX_alaasp_medium' since it already exists.
Ignoring reaction 'EX_alagln_medium' since it already exists.
Ignoring reaction 'EX_alaglu_medium' since it already exists.
Ignoring reaction 'EX_alagly_medium' since it already exists.
Ignoring reaction 'EX_alahis_medium' since it already exists.
Ignoring reaction 'EX_alaleu_medium' since it already exists.
Ignoring reaction 'EX_alathr_medium' since it already exists.
Ignoring reaction 'EX_ala_L_medium' since it already exists.
Ignoring reaction 'EX_h2o_medium' since it already exists.
Ignoring reaction 'EX_glc_D_medium' since it already exists.
Ignoring reaction 'EX_arbt_medium' since it already exists.
Ignoring reaction 'EX_arab

Note: no products in the objective function, adding biomass to it.


Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.
c:\Users\tobyl\miniconda3\envs\masterswork\Lib\site-packages\cobra\medium\boundary_types.py:97: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return most[0]
Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.
c:\Users\tobyl\miniconda3\envs\masterswork\Lib\site-packages\cobra\medium\boundary_types.py:97: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, in

Ignoring reaction 'EX_h_medium' since it already exists.
Ignoring reaction 'EX__2hyoxplac_medium' since it already exists.
Ignoring reaction 'EX_acnam_medium' since it already exists.
Ignoring reaction 'EX_ac_medium' since it already exists.
Ignoring reaction 'EX_adn_medium' since it already exists.
Ignoring reaction 'EX_adocbl_medium' since it already exists.
Ignoring reaction 'EX_h2o_medium' since it already exists.
Ignoring reaction 'EX_strch1_medium' since it already exists.
Ignoring reaction 'EX_glc_D_medium' since it already exists.
Ignoring reaction 'EX_strch2_medium' since it already exists.
Ignoring reaction 'EX_arab_L_medium' since it already exists.
Ignoring reaction 'EX_arg_L_medium' since it already exists.
Ignoring reaction 'EX_asn_L_medium' since it already exists.
Ignoring reaction 'EX_btn_medium' since it already exists.
Ignoring reaction 'EX_ca2_medium' since it already exists.
Ignoring reaction 'EX_cbl1_medium' since it already exists.
Ignoring reaction 'EX_cbl2_medi

Generated community model.


Name,ten_gutControl_model
Memory address,17da742e9c0
Number of metabolites,9709
Number of reactions,10500
Number of genes,2749
Number of groups,76
Objective expression,1.0*community_biomass - 1.0*community_biomass_reverse_44dc1
Compartments,"Escherichia_coli_UTI89_UPEC_c, Escherichia_coli_UTI89_UPEC_e, Escherichia_coli_UTI89_UPEC_medium, medium, fraction_reaction, Faecalibacterium_prausnitzii_M21_2_e, Faecalibacterium_prausnitzii_M21_2_c, Faecalibacterium_prausnitzii_M21_2_medium, Parabacteroides_distasonis_ATCC_8503_e, Parabacteroides_distasonis_ATCC_8503_c, Parabacteroides_distasonis_ATCC_8503_medium"


In [10]:
##### medium creation code


model_members = com_model_obj.member_models
single_org_model = model_members[0]
model = single_org_model.model
for reaction in model.exchanges:
    print(reaction.id)
    print(reaction)
    print(reaction.bounds)

EX_15dap(e)
EX_15dap(e): 15dap[e] --> 
(0.0, 1000.0)
EX_2ddglcn(e)
EX_2ddglcn(e): 2ddglcn[e] --> 
(0.0, 1000.0)
EX_2hyoxplac(e)
EX_2hyoxplac(e): 2hyoxplac[e] --> 
(0.0, 1000.0)
EX_4abut(e)
EX_4abut(e): 4abut[e] --> 
(0.0, 1000.0)
EX_4hbz(e)
EX_4hbz(e): 4hbz[e] <=> 
(-1.0, 1000.0)
EX_7dhcdchol(e)
EX_7dhcdchol(e): 7dhcdchol[e] --> 
(0.0, 1000.0)
EX_7ocholate(e)
EX_7ocholate(e): 7ocholate[e] --> 
(0.0, 1000.0)
EX_C02528(e)
EX_C02528(e): C02528[e] --> 
(0.0, 1000.0)
EX_Lcyst(e)
EX_Lcyst(e): Lcyst[e] --> 
(0.0, 1000.0)
EX_ac(e)
EX_ac(e): ac[e] <=> 
(-1.0, 1000.0)
EX_acac(e)
EX_acac(e): acac[e] --> 
(0.0, 1000.0)
EX_acald(e)
EX_acald(e): acald[e] --> 
(0.0, 1000.0)
EX_acgam(e)
EX_acgam(e): acgam[e] <=> 
(-1.0, 1000.0)
EX_adn(e)
EX_adn(e): adn[e] <=> 
(-1.0, 1000.0)
EX_adpcbl(e)
EX_adpcbl(e): adocbl[e] <=> 
(-1.0, 1000.0)
EX_akg(e)
EX_akg(e): akg[e] --> 
(0.0, 1000.0)
EX_ala_D(e)
EX_ala_D(e): ala_D[e] <=> 
(-1.0, 1000.0)
EX_ala_L(e)
EX_ala_L(e): ala_L[e] <=> 
(-1.0, 1000.0)
EX_alaasp(e)
EX_al

### Getting the Summary and Report for the model

In [9]:
com_model_obj.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
_4abz_medium,EX__4abz_medium,0.009289,7,0.05%
asn_L_medium,EX_asn_L_medium,0.2008,4,0.67%
asp_L_medium,EX_asp_L_medium,4.442,4,14.74%
ca2_medium,EX_ca2_medium,0.003096,0,0.00%
cl_medium,EX_cl_medium,0.003096,0,0.00%
cobalt2_medium,EX_cobalt2_medium,0.003096,0,0.00%
cu2_medium,EX_cu2_medium,0.003097,0,0.00%
cys_L_medium,EX_cys_L_medium,0.07615,3,0.19%
ddca_medium,EX_ddca_medium,0.2353,12,2.34%
etoh_medium,EX_etoh_medium,1.825,2,3.03%


- In report, need to check any unbalanced reactions that are not sink/demand (SINK, DM), Exchange (EX), Transport (T_ or TR_) or BIOMASS

In [10]:
com_model_obj.report()

Note: The model has more than 5000 reactions. Calculation of loops is skipped, as this would take some time. If needed, please run manually via .get_loops()
Name: ten_gutControl_model
------------------
Model overview
Model structure: fixed growth rate
# Metabolites: 12062
# Constraint (f-) Metabolites: 7585
# Model Metabolites: 4477
# Reactions: 13527
# Constraint (f-) Reactions: 7584
# Model Reactions: 5943
# Genes: 2164
# Members: 3
Members:
	Bacteroides_uniformis_ATCC_8492
	Bifidobacterium_adolescentis_ATCC_15703
	Faecalibacterium_prausnitzii_M21_2
Objective in direction max:
	1.0*community_biomass - 1.0*community_biomass_reverse_44dc1
------------------
Model quality
# Reactions unbalanced: 64
# Reactions able to carry flux without a medium: NaN


{'community_name': 'ten_gutControl_model',
 'model_structure': 'fixed growth rate',
 'num_metabolites': 12062,
 'num_f_metabolites': 7585,
 'num_model_metabolites': 4477,
 'num_reactions': 13527,
 'num_f_reactions': 7584,
 'num_model_reactions': 5943,
 'num_genes': 2164,
 'member_names': ['Bacteroides_uniformis_ATCC_8492',
  'Bifidobacterium_adolescentis_ATCC_15703',
  'Faecalibacterium_prausnitzii_M21_2'],
 'num_members': 3,
 'objective_expression': 1.0*community_biomass - 1.0*community_biomass_reverse_44dc1,
 'objective_direction': 'max',
 'unbalanced_reactions': {<Reaction Bacteroides_uniformis_ATCC_8492_DHNAOPT_Bacteroides_uniformis_ATCC_8492_c at 0x260b74319a0>: {'charge': 2.0},
  <Reaction Bacteroides_uniformis_ATCC_8492_DM_4HBA_Bacteroides_uniformis_ATCC_8492_c at 0x260b7431d90>: {'C': -7.0,
   'H': -8.0,
   'O': -2.0},
  <Reaction Bacteroides_uniformis_ATCC_8492_DM_AMOB_Bacteroides_uniformis_ATCC_8492_c at 0x260b7431d30>: {'C': -15.0,
   'H': -19.0,
   'N': -5.0,
   'O': -6.0,


- Checking if there are any thermodynamically infeasible cycles

In [11]:
# com_model_obj.get_loops()
# taking too long to run so I stopped it
# could be a problem and will have to 

## Step 3: Converting to fixed abundances as we are modelling specific conditions

In [12]:
# creating the abundance_dict
abundance_dict = com_model_obj.generate_equal_abundance_dict()
print(sum(abundance_dict.values()))
print(abundance_dict)

1.0
{'Bacteroides_uniformis_ATCC_8492': 0.3333333333333333, 'Bifidobacterium_adolescentis_ATCC_15703': 0.3333333333333333, 'Faecalibacterium_prausnitzii_M21_2': 0.3333333333333333}


In [13]:
# converting to fixed abundance model and then adding the abundances from abundance_dict
com_model_obj.convert_to_fixed_abundance()
com_model_obj.apply_fixed_abundance(abundance_dict)

# summarising the model
com_model_obj.summary()





Metabolite,Reaction,Flux,C-Number,C-Flux
Lcystin_medium,EX_Lcystin_medium,2.564,6,0.07%
_12dgr180_medium,EX__12dgr180_medium,0.1461,39,0.03%
_26dap_M_medium,EX__26dap_M_medium,0.5881,7,0.02%
_4abz_medium,EX__4abz_medium,0.1878,7,0.01%
acald_medium,EX_acald_medium,333.3,2,3.25%
alagln_medium,EX_alagln_medium,301.7,8,11.76%
alathr_medium,EX_alathr_medium,3.778,7,0.13%
amp_medium,EX_amp_medium,25.11,10,1.22%
arg_L_medium,EX_arg_L_medium,5.829,6,0.17%
asn_L_medium,EX_asn_L_medium,38.16,4,0.74%


In [ ]:
summary = com_model_obj.summary()

up_flux = summary.uptake_flux # how to get the uptake flux
sec_flux = summary.secretion_flux # how to get the secretion flux

# can get the objective value and status from this!
solution = com_model_obj.model.optimize()
print(solution.objective_value) # how to get the objective_value
print(solution.status) # How to get the status

print(up_flux.head())
print(sec_flux.head())


60.657421122172
optimal
                           flux             reaction        metabolite
EX_Lcystin_medium      2.563567    EX_Lcystin_medium    Lcystin_medium
EX__12dgr180_medium    0.146071  EX__12dgr180_medium  _12dgr180_medium
EX__26dap_M_medium     0.588134   EX__26dap_M_medium   _26dap_M_medium
EX__4abz_medium        0.187826      EX__4abz_medium      _4abz_medium
EX_acald_medium      333.333333      EX_acald_medium      acald_medium
                       flux               reaction          metabolite
EX_C02528_medium        0.0       EX_C02528_medium       C02528_medium
EX_Lcyst_medium         0.0        EX_Lcyst_medium        Lcyst_medium
EX_MGlcn100_medium      0.0     EX_MGlcn100_medium     MGlcn100_medium
EX_MGlcn100_rl_medium   0.0  EX_MGlcn100_rl_medium  MGlcn100_rl_medium
EX_MGlcn101_medium      0.0     EX_MGlcn101_medium     MGlcn101_medium


- Now, we can check all the potential exchange metabolic fluxes and cross-feeding interactions

In [14]:
MES = com_model_obj.potential_metabolite_exchanges()

In [15]:
MES.to_csv("mes_results.csv")